In [2]:
!pip install sentence-transformers tqdm pandas numpy chardet

In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import chardet
import re

file_path = "qwen_dialogue.csv"
with open(file_path, "rb") as f:
    raw_data = f.read(200000)
    result = chardet.detect(raw_data)
encoding = result["encoding"]
print(f"📄 Detected file encoding: {encoding}")


df = pd.read_csv(file_path, encoding=encoding)
print("✅ File loaded successfully!")
print("Columns in file:", df.columns.tolist())


def find_text_columns(columns):
    # find columns containing agent, speaker, or reply keywords
    candidates = [c for c in columns if re.search(r'(agent|reply|text|utter|speaker)', c, re.I)]
    # keep only object (text-like) columns
    return candidates[:2] if len(candidates) >= 2 else columns[:2]

text_cols = find_text_columns(df.columns)
if len(text_cols) < 2:
    raise ValueError("❌ Could not detect two text columns. Please check your CSV manually.")
    
col1, col2 = text_cols[:2]
print(f" Using columns for comparison: {col1} and {col2}")

model = SentenceTransformer('all-MiniLM-L6-v2')

scores = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Computing Agreement Scores..."):
    text1, text2 = str(row[col1]), str(row[col2])
    emb1, emb2 = model.encode(text1), model.encode(text2)
    cosine_sim = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    scores.append(cosine_sim)

df["agreement_score"] = scores

df.to_csv("qwen_dialogue_agreement.csv", index=False, encoding="utf-8-sig")
print(f"✅ Agreement scores saved to qwen_dialogue_agreement_score.csv ({len(df)} rows)")

📄 Detected file encoding: Windows-1252
✅ File loaded successfully!
Columns in file: ['topic_id', 'topic', 'category', 'contentiousness', 'statement', 'pair_type', 'agentA_row', 'agentB_row', 'prefA', 'prefB', 'dialogue']
 Using columns for comparison: agentA_row and agentB_row


Computing Agreement Scores...: 100%|████████| 144/144 [00:01<00:00, 80.90it/s]

✅ Agreement scores saved to qwen_dialogue_agreement_score.csv (144 rows)
